In [1]:
import sys
sys.path.append('/home/jiajunb/neural-dimension-reduction')

In [2]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import functional as F
from src.models.distance_modeling import SurveyorDataSet, Surveyor, thesis_kl_div_add_mse_loss

import copy

from sklearn.metrics import classification_report

torch.manual_seed(0)

In [17]:
def far_func2(sorted_dist: torch.tensor, indices: torch.tensor):
     return sorted_dist[:, 21:], indices[:, 21:]
#     n, d = sorted_dist.shape
    
#     tmp1, tmp2 = list(), list()
#     for i in range(n):
#         idx = torch.randint(low=(d - 21), high=(d - 1), size=(1, 20))
#         tmp1.append(sorted_dist[i][idx])
#         tmp2.append(indices[i][idx])
#     return torch.cat(tmp1, dim=0), torch.cat(tmp2, dim=0)

def close_func2(sorted_dist: torch.tensor, indices: torch.tensor):
    return sorted_dist[:, 1:21], indices[:, 1:21]

test_dataset = SurveyorDataSet.from_df('/home/jiajunb/neural-dimension-reduction/data/processed/sample/dev.csv', close_func2, far_func2)

In [18]:

def val_one_epoch(val_loader, model):
    model.eval()
    loss_fn1 = nn.CrossEntropyLoss()
    loss_fn2 = thesis_kl_div_add_mse_loss
    preds_list = list()
    labels_list = list()
    val_xentropy_loss = 0.
    val_thesis_loss = 0.
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            x1, x2, labels, q = batch
            x1, x2, q = x1.to(device), x2.to(device), q.to(device)
            logits, p, out1, out2 = model(x1, x2, q, labels=None)
            preds = torch.argmax(F.softmax(logits, dim=1), dim=1)
            preds_list.append(preds.cpu())
            labels_list.append(labels.cpu())
            labels = labels.to(device)
            val_xentropy_loss += loss_fn1(logits, labels).item()
            val_thesis_loss += loss_fn2(p, q).item()
    y_preds = torch.cat(preds_list)
    y_golds = torch.cat(labels_list)
    accuracy = float((y_preds == y_golds).sum().item()) / len(y_preds)
    print(classification_report(y_golds, y_preds))
    return val_xentropy_loss / len(y_preds), val_thesis_loss / len(y_preds), accuracy

In [19]:
device = torch.device('cuda:1')

In [20]:
best_model = Surveyor()
best_model.load_state_dict(torch.load('../saves/surveyor.on.multi-neg.sample.100')['best_model'])
best_model.eval()

Surveyor(
  (encoder): Sequential(
    (bn0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (fc0): Linear(in_features=200, out_features=500, bias=True)
    (bn1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (fc1): Linear(in_features=500, out_features=100, bias=True)
    (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (fc2): Linear(in_features=100, out_features=20, bias=True)
    (bn3): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (fc3): Linear(in_features=20, out_features=20, bias=True)
    (bn4): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu4): ReLU(inplace=True)
    (fc4): Linear(in_features=20, out_features=20, bias=True)
    (bn5): BatchNorm1d(20, eps=1

In [21]:
best_model = best_model.to(device)

In [22]:
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1000, pin_memory=True)
avg_xentropy_loss, avg_thesis_loss, val_accuracy = val_one_epoch(test_loader, best_model)

print(f'val_cross_entropy_loss: {avg_xentropy_loss:.4f} '
      f'val_thesis_loss: {avg_thesis_loss:.4f} '
      f'val_accuracy: {val_accuracy:.4f} ')

              precision    recall  f1-score   support

           0       1.00      0.46      0.63    429570
           1       0.05      1.00      0.10     13320

    accuracy                           0.47    442890
   macro avg       0.53      0.73      0.36    442890
weighted avg       0.97      0.47      0.61    442890

val_cross_entropy_loss: 0.0007 val_thesis_loss: 0.3037 val_accuracy: 0.4715 


In [23]:
len(test_dataset)

442890

In [24]:
torch.sum(test_dataset.labels)

tensor(13320)

In [25]:
len(test_dataset.labels)

442890

In [26]:
torch.sum(1 - test_dataset.labels)

tensor(429570)

In [27]:
len(test_dataset)

442890